In [1]:
import yfinance as yf
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
from gpflow.kernels import Matern32, Kernel
from gpflow.models import GPR
from gpflow import set_trainable
from sklearn.preprocessing import StandardScaler
import vectorbt as vbt
from datetime import datetime, timedelta
import ruptures as rpt
import warnings

warnings.filterwarnings("ignore", category=FutureWarning)

# Define the custom ChangePointKernel for Gaussian Process Regression
class ChangePointKernel(Kernel):
    def __init__(self, base_kernel, changepoints):
        super().__init__()
        self.base_kernel = base_kernel
        # changepoints: list of indices where a regime change is detected.
        self.changepoints = changepoints

    def get_region(self, X):
        cp = tf.constant(self.changepoints, dtype=X.dtype)
        regions = tf.searchsorted(cp, X[:, 0], side='right')
        return regions

    def K(self, X, X2=None):
        if X2 is None:
            X2 = X
        regions_X = self.get_region(X)
        regions_X2 = self.get_region(X2)
        regions_equal = tf.cast(tf.equal(tf.expand_dims(regions_X, 1), tf.expand_dims(regions_X2, 0)), X.dtype)
        base_cov = self.base_kernel.K(X, X2)
        return base_cov * regions_equal

    def K_diag(self, X):
        return self.base_kernel.K_diag(X)

def fetch_and_process_data(tickers, start_date, end_date):
    """Fetch data for multiple tickers and return price and returns DataFrames."""
    price_data = {}
    returns_data = {}
    
    # Add buffer days to ensure we have enough data
    start_date_obj = datetime.strptime(start_date, '%Y-%m-%d') - timedelta(days=30)
    start_date_buffer = start_date_obj.strftime('%Y-%m-%d')
    
    for ticker in tickers:
        try:
            # Use buffered start date to get more data
            data = yf.download(ticker, start=start_date_buffer, end=end_date)
            if data.empty:
                print(f"No data found for {ticker} between {start_date_buffer} and {end_date}")
                continue
                
            # Prefer "Adj Close" if available, otherwise "Close"
            if "Adj Close" in data.columns:
                price_series = data["Adj Close"].copy()
            else:
                price_series = data["Close"].copy()
                
            if isinstance(price_series, pd.DataFrame):
                price_series = price_series.squeeze()
                
            price_series.name = ticker
            
            # Check data quality
            missing_values = price_series.isna().sum()
            if missing_values > 0:
                print(f"Warning: {ticker} has {missing_values} missing values. Filling with ffill method.")
                price_series = price_series.fillna(method='ffill').fillna(method='bfill')
            
            returns = price_series.pct_change().dropna()
            
            # Add validation check for zero or near-zero returns
            zero_returns = (returns.abs() < 1e-8).sum()
            if zero_returns > len(returns) * 0.3:  # If more than 30% are zero returns
                print(f"Warning: {ticker} has {zero_returns} zero or near-zero returns (possible stale data)")
            
            # Standardize returns
            scaler = StandardScaler()
            std_returns = scaler.fit_transform(returns.values.reshape(-1, 1))
            valid_index = returns.index
            
            # Filter to the actual date range needed
            actual_start = datetime.strptime(start_date, '%Y-%m-%d')
            final_index = valid_index[valid_index >= pd.Timestamp(actual_start)]
            
            # Only keep data if we have enough for the given date range
            if len(final_index) > 0:
                price_data[ticker] = price_series
                returns_data[ticker] = pd.Series(
                    std_returns.flatten(), index=valid_index, name=f"{ticker}_stdret"
                )
            else:
                print(f"Not enough valid data for {ticker} in the specified date range after preprocessing")
                
        except Exception as e:
            print(f"Error fetching {ticker}: {e}")
    
    # Convert to DataFrames
    prices_df = pd.DataFrame({ticker: series for ticker, series in price_data.items()})
    returns_df = pd.DataFrame({ticker: series for ticker, series in returns_data.items()})
    
    # Filter to the actual date range needed
    actual_start = datetime.strptime(start_date, '%Y-%m-%d')
    prices_df = prices_df[prices_df.index >= pd.Timestamp(actual_start)]
    returns_df = returns_df[returns_df.index >= pd.Timestamp(actual_start)]
    
    return prices_df, returns_df

def detect_changepoints(returns, lookback_window, min_size=10):
    """Detect changepoints in return series with minimum segment size."""
    if len(returns) < min_size * 2:
        print(f"Warning: Series too short for changepoint detection ({len(returns)} points)")
        return []  # Return empty list if series is too short
        
    # Use ruptures to detect changepoints using an RBF model with a minimum segment size
    algo = rpt.Pelt(model="rbf", min_size=min_size).fit(returns.values)
    # Use the lookback_window as penalty; this is a simplified proxy for CPD LBW.
    result = algo.predict(pen=lookback_window)
    
    # Remove the last element which is just the length of the series
    if result and result[-1] == len(returns):
        result = result[:-1]
        
    # Print number of changepoints detected
    print(f"Detected {len(result)} changepoints in series of length {len(returns)}")
    
    return result

def generate_signals(returns, changepoints, train_ratio, epochs, 
                     dropout_rate, lstm_hidden_units, learning_rate, batch_size):
    """Generate trading signals using GPR+LSTM model."""
    # Use time index as a feature.
    X = np.arange(len(returns), dtype=np.float64).reshape(-1, 1)
    y = returns.values.reshape(-1, 1)
    
    # Use Gaussian Process Regression with the ChangePointKernel for trend extraction.
    base_kernel = Matern32()
    kernel = ChangePointKernel(base_kernel, changepoints)
    gpr = GPR(data=(X, y), kernel=kernel)
    set_trainable(gpr.likelihood.variance, False)
    trend = gpr.predict_f(X)[0].numpy().flatten()
    
    # Combine returns and trend to form a feature set.
    features = np.hstack([returns.values.reshape(-1, 1), trend.reshape(-1, 1)])
    
    # Ensure we have enough data for training
    if len(features) < 30:  # Minimum reasonable number for time series
        print("Warning: Not enough data points for effective training")
        # Return neutral signals if data is insufficient
        return np.ones(len(returns))
    
    # Ensure train_ratio doesn't create too small of a training set
    min_train_size = 30
    if int(len(features) * train_ratio) < min_train_size:
        train_ratio = max(0.5, min_train_size / len(features))
        print(f"Adjusted train_ratio to {train_ratio:.2f} to ensure minimum training size")
    
    split_index = int(len(features) * train_ratio)
    train_features = features[:split_index]
    train_labels = returns.values[:split_index]
    
    # Reshape features for LSTM input.
    train_features = train_features.reshape((train_features.shape[0], 1, train_features.shape[1]))
    
    # Add early stopping to prevent overfitting
    early_stop = tf.keras.callbacks.EarlyStopping(
        monitor='loss', patience=5, restore_best_weights=True
    )
    
    # Build the LSTM model.
    model = Sequential([
        LSTM(lstm_hidden_units, return_sequences=True,
             input_shape=(train_features.shape[1], train_features.shape[2]),
             activation='tanh',  # Explicit activation
             recurrent_activation='sigmoid'),  # Explicit recurrent activation
        Dropout(dropout_rate),
        LSTM(int(lstm_hidden_units / 2)),
        Dense(1, activation="tanh")
    ])
    model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=learning_rate), loss="mse")
    
    # Train the model with early stopping
    model.fit(
        train_features, train_labels, 
        epochs=epochs, batch_size=min(batch_size, len(train_features)), 
        verbose=0,
        callbacks=[early_stop]
    )
    
    # Predict signals for all features.
    all_features = features.reshape((features.shape[0], 1, features.shape[1]))
    predicted_signals = model.predict(all_features, verbose=0).flatten()
    
    # Analyze signal distribution
    signal_mean = np.mean(predicted_signals)
    signal_std = np.std(predicted_signals)
    signal_min = np.min(predicted_signals)
    signal_max = np.max(predicted_signals)
    print(f"Signal stats - Mean: {signal_mean:.4f}, StdDev: {signal_std:.4f}, Min: {signal_min:.4f}, Max: {signal_max:.4f}")
    
    # Convert signals to binary (-1 or 1) for direction only
    positions = np.sign(predicted_signals)
    positions[positions == 0] = 1  # Convert any zeros to 1 (long)
    
    # Ensure we're getting a good mix of signals (at least some variation)
    long_pct = np.sum(positions > 0) / len(positions) * 100
    short_pct = np.sum(positions < 0) / len(positions) * 100
    print(f"Position distribution: {long_pct:.1f}% long, {short_pct:.1f}% short")
    
    # If signals are too one-sided (>95% same direction), add a warning
    if long_pct > 95 or short_pct > 95:
        print("Warning: Signals are very one-sided, might indicate poor model fit")
    
    return positions

def backtest_strategy_single_asset(price_series, positions):
    """Backtest strategy for a single asset with proper position alignment."""
    if isinstance(price_series, pd.DataFrame):
        price_series = price_series.squeeze()
    
    # Ensure positions align with price_series
    common_idx = price_series.index[:len(positions)]
    
    # Verify we have enough data points to proceed
    if len(common_idx) < 10:
        print(f"Warning: Not enough aligned data points for {price_series.name}")
        return None
    
    # Create properly aligned series
    aligned_prices = price_series.loc[common_idx]
    aligned_positions = pd.Series(positions, index=common_idx, name=price_series.name)
    
    # Debug info
    print(f"Backtesting {price_series.name}: {len(aligned_prices)} aligned price points with {len(aligned_positions)} position signals")
    
    # Convert positions to entries/exits
    entries = aligned_positions > 0
    exits = aligned_positions < 0
    
    # Count actual trading events
    entry_count = entries.sum()
    exit_count = exits.sum()
    print(f"Trading activity: {entry_count} entries, {exit_count} exits")
    
    # Ensure we have at least some trading activity
    if entry_count == 0 or exit_count == 0:
        print(f"Warning: Insufficient trading signals for {price_series.name}")
        if entry_count == 0 and exit_count == 0:
            # No trading signals at all, return None
            return None
    
    try:
        # Only use supported parameters for vectorbt Portfolio
        pf = vbt.Portfolio.from_signals(
            close=aligned_prices,
            entries=entries,
            exits=exits,
            size=1.0,  # Always use full size
            freq="1D",
            fees=0.001,
            direction='both'  # Allow both long and short
            # Removed unsupported parameters
        )
        
        # Verify position sizes
        max_gross_exposure = pf.gross_exposure().max()
        min_gross_exposure = pf.gross_exposure().min()
        mean_gross_exposure = pf.gross_exposure().mean()
        
        print(f"Exposure for {price_series.name}: Min={min_gross_exposure:.2%}, Mean={mean_gross_exposure:.2%}, Max={max_gross_exposure:.2%}")
        
        return pf
    except Exception as e:
        print(f"Error in backtesting {price_series.name}: {e}")
        return None

def backtest_equal_weight_portfolio(prices_df, signals_dict):
    """Backtest a portfolio with equal weights across assets."""
    # Create a dictionary to store individual asset performance
    asset_results = {}
    
    for ticker in prices_df.columns:
        if ticker in signals_dict:
            # Get price series and signals for this ticker
            price_series = prices_df[ticker]
            positions = signals_dict[ticker]
            
            print(f"\nProcessing {ticker}:")
            print(f"  - Price series: {len(price_series)} points from {price_series.index[0].date()} to {price_series.index[-1].date()}")
            print(f"  - Position signals: {len(positions)} points")
            
            # Backtest the individual asset
            pf = backtest_strategy_single_asset(price_series, positions)
            if pf is not None:
                asset_results[ticker] = pf
            else:
                print(f"Skipping {ticker} due to backtesting issues")
    
    # Calculate the equal weight allocation (1/n for each asset)
    n_assets = len(asset_results)
    if n_assets == 0:
        print("No valid assets to create portfolio!")
        return None, {}
        
    weight_per_asset = 1.0 / n_assets
    print(f"\nCreating equal-weight portfolio with {n_assets} assets, {weight_per_asset:.2%} weight each")
    
    # Create a combined DataFrame for returns
    combined_returns = pd.DataFrame()
    
    for ticker, pf in asset_results.items():
        returns = pf.returns()
        if not returns.empty:
            combined_returns[ticker] = returns
    
    # Calculate equal-weighted portfolio returns
    if not combined_returns.empty:
        # Replace NaN with 0 for calculation
        combined_returns.fillna(0, inplace=True)
        portfolio_returns = (combined_returns * weight_per_asset).sum(axis=1)
        
        # Calculate cumulative returns
        portfolio_cumulative = (1 + portfolio_returns).cumprod()
        
        # Enhanced portfolio stats
        sharpe = portfolio_returns.mean() / portfolio_returns.std() * np.sqrt(252) if len(portfolio_returns) > 0 else 0
        total_return = portfolio_cumulative.iloc[-1] - 1 if len(portfolio_cumulative) > 0 else 0
        max_dd = (portfolio_cumulative / portfolio_cumulative.cummax() - 1).min() if len(portfolio_cumulative) > 0 else 0
        win_rate = np.sum(portfolio_returns > 0) / len(portfolio_returns) if len(portfolio_returns) > 0 else 0
        
        # Calculate annualized return
        days = (portfolio_returns.index[-1] - portfolio_returns.index[0]).days
        years = days / 365.0
        annualized_return = (1 + total_return) ** (1 / years) - 1 if years > 0 else 0
        
        portfolio_stats = {
            "total_return": total_return,
            "annualized_return": annualized_return,
            "sharpe_ratio": sharpe,
            "max_drawdown": max_dd,
            "win_rate": win_rate,
            "num_days": days
        }
        
        # Create mock portfolio object
        mock_portfolio = type('obj', (object,), {
            'stats': lambda: portfolio_stats,
            'returns': lambda: portfolio_returns,
            'cumulative_returns': lambda: portfolio_cumulative
        })
        
        print(f"\nPortfolio Performance:")
        print(f"  - Total Return: {total_return:.2%}")
        print(f"  - Annualized Return: {annualized_return:.2%}")
        print(f"  - Sharpe Ratio: {sharpe:.2f}")
        print(f"  - Max Drawdown: {max_dd:.2%}")
        print(f"  - Win Rate: {win_rate:.2%}")
        
        return mock_portfolio, asset_results
    else:
        print("No valid returns data to create portfolio!")
        return None, {}

def main():
    # Fixed hyperparameters.
    params = {
        'batch_size': 64,  # Reduced from 128 to avoid overfitting with small datasets
        'dropout_rate': 0.25,  # Increased from 0.2 for better regularization
        'learning_rate': 0.001,
        'lookback_window': 21,
        'lstm_hidden_units': 40,
        'epochs': 50,
        'train_ratio': 0.8
    }
    
    # Updated dictionary of top cryptocurrencies by year with -USD suffix
    top_cryptos = {
        "2019": ["BTC-USD", "ETH-USD", "XRP-USD", "BCH-USD", "EOS-USD", "LTC-USD", "XLM-USD", "ADA-USD", "TRX-USD", "BSV-USD"],
        "2020": ["BTC-USD", "ETH-USD", "XRP-USD", "BCH-USD", "LTC-USD", "EOS-USD", "BNB-USD", "BSV-USD", "ADA-USD", "XTZ-USD"],
        "2021": ["BTC-USD", "ETH-USD", "XRP-USD", "LTC-USD", "BCH-USD", "ADA-USD", "DOT1-USD", "LINK-USD", "BNB-USD", "XLM-USD"],
        "2022": ["BTC-USD", "ETH-USD", "BNB-USD", "SOL-USD", "ADA-USD", "XRP-USD", "DOT1-USD", "LUNA1-USD", "AVAX-USD", "DOGE-USD"],  # Fixed LUNA -> LUNA1-USD
        "2023": ["BTC-USD", "ETH-USD", "BNB-USD", "XRP-USD", "ADA-USD", "DOGE-USD", "MATIC-USD", "DOT1-USD", "LTC-USD", "SHIB-USD"],
        "2024": ["BTC-USD", "ETH-USD", "BNB-USD", "XRP-USD", "ADA-USD", "DOGE-USD", "SOL-USD", "DOT1-USD", "LTC-USD", "AVAX-USD"],
        "2025": ["BTC-USD", "ETH-USD", "BNB-USD", "XRP-USD", "ADA-USD", "DOGE-USD", "SOL-USD", "DOT1-USD", "LTC-USD", "AVAX-USD"]
    }

    # Define periods (one per year)
    periods = []
    for year in range(2019, 2025):
        period_label = str(year)
        start_date = f"{year}-01-01"
        end_date = f"{year}-12-31"
        periods.append((period_label, start_date, end_date))
    # YTD 2025 using current date
    periods.append(("2025_YTD", "2025-01-01", "2025-03-07"))

    results = []

    # Loop over each period and backtest the respective top cryptocurrencies for that year
    for period_label, start_date, end_date in periods:
        year = period_label.split("_")[0]  # Extract year from period label
        cryptos_for_year = top_cryptos.get(year, [])
        
        print(f"\n{'='*80}")
        print(f"Processing period {period_label} ({start_date} to {end_date})")
        print(f"Cryptocurrencies: {', '.join(cryptos_for_year)}")
        print(f"{'='*80}")
        
        # Fetch all data for the period
        prices_df, returns_df = fetch_and_process_data(cryptos_for_year, start_date, end_date)
        
        if prices_df.empty or returns_df.empty:
            print(f"No data available for period {period_label}")
            continue
        
        # Debug info about the data
        print(f"\nData summary:")
        print(f"  - Price data: {len(prices_df)} days from {prices_df.index[0].date() if not prices_df.empty else 'N/A'} to {prices_df.index[-1].date() if not prices_df.empty else 'N/A'}")
        print(f"  - Returns data: {len(returns_df)} days")
        print(f"  - Available tickers: {', '.join(prices_df.columns)}")
        
        # Generate signals for each cryptocurrency
        signals_dict = {}
        
        for crypto in cryptos_for_year:
            if crypto in returns_df.columns:
                try:
                    print(f"\n{'-'*60}")
                    print(f"Processing {crypto} for period {period_label}")
                    
                    returns = returns_df[crypto]
                    if len(returns) < params['lookback_window']:
                        print(f"Not enough data for {crypto} in period {period_label}. Skipping.")
                        continue
                    
                    print(f"Detecting changepoints for {crypto}...")
                    changepoints = detect_changepoints(returns, lookback_window=params['lookback_window'])
                    
                    print(f"Generating trading signals for {crypto}...")
                    positions = generate_signals(
                        returns, changepoints,
                        train_ratio=params['train_ratio'],
                        epochs=params['epochs'],
                        dropout_rate=params['dropout_rate'],
                        lstm_hidden_units=params['lstm_hidden_units'],
                        learning_rate=params['learning_rate'],
                        batch_size=params['batch_size']
                    )
                    
                    # Make sure positions match the length of the returns
                    signals_dict[crypto] = positions
                    print(f"Generated {len(positions)} position signals for {crypto}")
                    
                except Exception as e:
                    print(f"Error generating signals for {crypto}: {e}")
                    import traceback
                    traceback.print_exc()
        
        # Backtest with equal weight allocation
        portfolio, asset_portfolios = backtest_equal_weight_portfolio(prices_df, signals_dict)
        
        if portfolio:
            # Record main portfolio results
            portfolio_stats = portfolio.stats()
            result = {
                "period": period_label,
                "strategy": "Equal Weight Portfolio",
                "num_assets": len(asset_portfolios),
                **{key: str(value) if pd.isna(value) else value for key, value in portfolio_stats.items()}
            }
            results.append(result)
            
            # Also record individual asset results
            for crypto, pf in asset_portfolios.items():
                if hasattr(pf, 'stats'):
                    try:
                        asset_stats = pf.stats()
                        asset_result = {
                            "period": period_label,
                            "crypto": crypto,
                            "strategy": "Individual Asset",
                            **{key: str(value) if pd.isna(value) else value for key, value in asset_stats.items()}
                        }
                        results.append(asset_result)
                    except Exception as e:
                        print(f"Error extracting stats for {crypto}: {e}")

    # Save all summary results into a DataFrame.
    if results:  # Check if we have results before creating DataFrame
        results_df = pd.DataFrame(results)
        print("\nBacktest Results:")
        print(results_df)
        
        # Save the DataFrame to a CSV file.
        results_df.to_csv("Backtested/equal_weight_8.csv", index=False)
        
        # Calculate and display yearly performance comparison
        portfolio_results = results_df[results_df["strategy"] == "Equal Weight Portfolio"]
        if not portfolio_results.empty:
            print("\nEqual Weight Portfolio Performance by Year:")
            yearly_perf = portfolio_results[["period", "total_return", "sharpe_ratio", "max_drawdown"]]
            print(yearly_perf)
    else:
        print("\nNo results generated. Check the data and parameters.")

if __name__ == "__main__":
    main()


Processing period 2019 (2019-01-01 to 2019-12-31)
Cryptocurrencies: BTC-USD, ETH-USD, XRP-USD, BCH-USD, EOS-USD, LTC-USD, XLM-USD, ADA-USD, TRX-USD, BSV-USD
YF.download() has changed argument auto_adjust default to True


[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed



Data summary:
  - Price data: 364 days from 2019-01-01 to 2019-12-30
  - Returns data: 364 days
  - Available tickers: BTC-USD, ETH-USD, XRP-USD, BCH-USD, EOS-USD, LTC-USD, XLM-USD, ADA-USD, TRX-USD, BSV-USD

------------------------------------------------------------
Processing BTC-USD for period 2019
Detecting changepoints for BTC-USD...
Detected 0 changepoints in series of length 364
Generating trading signals for BTC-USD...


/opt/homebrew/lib/python3.11/site-packages/keras/src/layers/rnn/rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Signal stats - Mean: -0.0176, StdDev: 0.5859, Min: -0.9942, Max: 0.9981
Position distribution: 45.3% long, 54.7% short
Generated 364 position signals for BTC-USD

------------------------------------------------------------
Processing ETH-USD for period 2019
Detecting changepoints for ETH-USD...
Detected 0 changepoints in series of length 364
Generating trading signals for ETH-USD...
Signal stats - Mean: -0.0088, StdDev: 0.5982, Min: -0.9973, Max: 0.9968
Position distribution: 47.0% long, 53.0% short
Generated 364 position signals for ETH-USD

------------------------------------------------------------
Processing XRP-USD for period 2019
Detecting changepoints for XRP-USD...
Detected 0 changepoints in series of length 364
Generating trading signals for XRP-USD...
Signal stats - Mean: -0.0196, StdDev: 0.5808, Min: -0.9802, Max: 0.9983
Position distribution: 46.2% long, 53.8% short
Generated 364 position signals for XRP-USD

------------------------------------------------------------
Pr

...

No data found for DOT1-USD between 2023-12-02 and 2024-12-31

Data summary:
  - Price data: 365 days from 2024-01-01 to 2024-12-30
  - Returns data: 365 days
  - Available tickers: BTC-USD, ETH-USD, BNB-USD, XRP-USD, ADA-USD, DOGE-USD, SOL-USD, LTC-USD, AVAX-USD

------------------------------------------------------------
Processing BTC-USD for period 2024
Detecting changepoints for BTC-USD...


/opt/homebrew/lib/python3.11/site-packages/keras/src/layers/rnn/rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Detected 0 changepoints in series of length 365
Generating trading signals for BTC-USD...
Signal stats - Mean: -0.0242, StdDev: 0.6409, Min: -0.9883, Max: 0.9964
Position distribution: 46.8% long, 53.2% short
Generated 365 position signals for BTC-USD

------------------------------------------------------------
Processing ETH-USD for period 2024
Detecting changepoints for ETH-USD...
Detected 0 changepoints in series of length 365
Generating trading signals for ETH-USD...
Signal stats - Mean: -0.0145, StdDev: 0.6518, Min: -0.9937, Max: 0.9977
Position distribution: 49.3% long, 50.7% short
Generated 365 position signals for ETH-USD

------------------------------------------------------------
Processing BNB-USD for period 2024
Detecting changepoints for BNB-USD...
Detected 0 changepoints in series of length 365
Generating trading signals for BNB-USD...
Signal stats - Mean: -0.0444, StdDev: 0.6439, Min: -0.9922, Max: 0.9990
Position distribution: 46.3% long, 53.7% short
Generated 365 pos

/var/folders/b6/28cdp6ds16786vjhx3pz3jlr0000gn/T/ipykernel_84656/3910470318.py:341: RuntimeWarning: invalid value encountered in scalar power
  annualized_return = (1 + total_return) ** (1 / years) - 1 if years > 0 else 0
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed

1 Failed download:
['DOT1-USD']: YFPricesMissingError('possibly delisted; no price data found  (1d 2024-12-02 -> 2025-03-07)')
[*********************100%***********************]  1 of 1 completed
[*********************100%***

No data found for DOT1-USD between 2024-12-02 and 2025-03-07

Data summary:
  - Price data: 65 days from 2025-01-01 to 2025-03-06
  - Returns data: 65 days
  - Available tickers: BTC-USD, ETH-USD, BNB-USD, XRP-USD, ADA-USD, DOGE-USD, SOL-USD, LTC-USD, AVAX-USD

------------------------------------------------------------
Processing BTC-USD for period 2025_YTD
Detecting changepoints for BTC-USD...
Detected 0 changepoints in series of length 65
Generating trading signals for BTC-USD...
Signal stats - Mean: 0.0470, StdDev: 0.1566, Min: -0.3224, Max: 0.5602
Position distribution: 66.2% long, 33.8% short
Generated 65 position signals for BTC-USD

------------------------------------------------------------
Processing ETH-USD for period 2025_YTD
Detecting changepoints for ETH-USD...
Detected 0 changepoints in series of length 65
Generating trading signals for ETH-USD...
Signal stats - Mean: 0.0219, StdDev: 0.1433, Min: -0.3866, Max: 0.4448
Position distribution: 56.9% long, 43.1% short
Gener

In [6]:
df_backtest = pd.read_csv("Backtested/equal_weight_8.csv")

In [7]:
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
df_backtest

,period,strategy,num_assets,total_return,annualized_return,sharpe_ratio,max_drawdown,win_rate,num_days,crypto,Start,End,Period,Start Value,End Value,Total Return [%],Benchmark Return [%],Max Gross Exposure [%],Total Fees Paid,Max Drawdown [%],Max Drawdown Duration,Total Trades,Total Closed Trades,Total Open Trades,Open Trade PnL,Win Rate [%],Best Trade [%],Worst Trade [%],Avg Winning Trade [%],Avg Losing Trade [%],Avg Winning Trade Duration,Avg Losing Trade Duration,Profit Factor,Expectancy,Sharpe Ratio,Calmar Ratio,Omega Ratio,Sortino Ratio
0,2019,Equal Weight Portfolio,10.0,-0.403937,-4.056338e-01,-0.528959,-0.617004,0.469780,363.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2019,Individual Asset,NaN,NaN,NaN,NaN,NaN,NaN,NaN,BTC-USD,2019-01-01,2019-12-30,364 days,100.0,-301.638121,-401.638121,89.747811,100.000000,15.788835,625.238945,362 days,5.0,4.0,1.0,-1.427600e+02,0.000000,-0.376193,-6.224908,NaN,-1.968167,NaT,1 days 06:00:00,0.000000,-6.471953e+01,-1.306177,NaN,0.444649,-1.314306
2,2019,Individual Asset,NaN,NaN,NaN,NaN,NaN,NaN,NaN,ETH-USD,2019-01-01,2019-12-30,364 days,100.0,250.725296,150.725296,-5.813068,2250.405634,58.621382,78.198040,127 days,186.0,185.0,1.0,-1.326335e-01,32.432432,23.611161,-9.071376,6.103094,-2.171967,3 days 04:48:00,1 days 08:49:55.200000,1.345704,8.154483e-01,1.281170,1.935589,1.294362,2.604012
3,2019,Individual Asset,NaN,NaN,NaN,NaN,NaN,NaN,NaN,XRP-USD,2019-01-01,2019-12-30,364 days,100.0,100.019439,0.019439,-46.686005,0.474897,0.116652,0.204059,175 days,188.0,187.0,1.0,-1.944740e-04,33.689840,28.723192,-9.188114,4.213775,-2.103177,3 days 01:08:34.285714285,1 days 09:05:48.387096774,1.023580,1.049911e-04,0.082105,0.095523,1.013389,0.131776
4,2019,Individual Asset,NaN,NaN,NaN,NaN,NaN,NaN,NaN,BCH-USD,2019-01-01,2019-12-30,364 days,100.0,-4.822299,-104.822299,27.023918,100.000000,35.743189,103.570531,226 days,95.0,94.0,1.0,4.367746e+00,28.723404,70.684594,-13.411579,8.863750,-3.278497,3 days 08:00:00,1 days 08:35:49.253731343,0.750449,-1.161596e+00,-1.134171,NaN,0.708592,-1.215930
5,2019,Individual Asset,NaN,NaN,NaN,NaN,NaN,NaN,NaN,EOS-USD,2019-01-01,2019-12-30,364 days,100.0,100.263282,0.263282,-0.675378,8.471982,1.529286,1.942002,97 days,194.0,193.0,1.0,-2.645544e-03,30.569948,35.625649,-15.609205,6.491233,-2.962485,3 days 02:26:26.440677966,1 days 08:14:19.701492537,1.016379,1.377860e-03,0.081905,0.135945,1.013881,0.127072
6,2019,Individual Asset,NaN,NaN,NaN,NaN,NaN,NaN,NaN,LTC-USD,2019-01-01,2019-12-30,364 days,100.0,220.288814,120.288814,33.669261,9059.055589,23.977170,18.826215,40 days,174.0,173.0,1.0,-4.274734e-02,36.994220,51.689078,-12.319170,6.945096,-2.536105,3 days 07:30:00,1 days 09:14:51.743119266,1.639380,6.955582e-01,1.884266,6.414847,1.366293,3.382568
7,2019,Individual Asset,NaN,NaN,NaN,NaN,NaN,NaN,NaN,XLM-USD,2019-01-01,2019-12-30,364 days,100.0,100.140879,0.140879,-60.360562,0.141411,0.030571,0.039066,77 days,176.0,175.0,1.0,-4.595400e-05,37.142857,40.713323,-10.774890,6.565665,-2.371343,3 days 08:29:32.307692307,1 days 07:38:10.909090909,1.615253,8.052864e-04,1.860304,3.616084,1.337424,3.244456
8,2019,Individual Asset,NaN,NaN,NaN,NaN,NaN,NaN,NaN,ADA-USD,2019-01-01,2019-12-30,364 days,100.0,100.015921,0.015921,-21.042607,0.097724,0.020085,0.049882,194 days,184.0,183.0,1.0,-3.359400e-05,31.693989,24.790683,-9.967786,5.777548,-2.675642,3 days 05:47:35.172413793,1 days 09:36:00,1.087361,8.718110e-05,0.284404,0.320041,1.044297,0.449056
9,2019,Individual Asset,NaN,NaN,NaN,NaN,NaN,NaN,NaN,TRX-USD,2019-01-01,2019-12-30,364 days,100.0,100.020917,0.020917,-31.496347,0.038555,0.008298,0.013425,136 days,186.0,185.0,1.0,-1.340000e-05,36.756757,27.276977,-15.593502,5.800813,-2.764090,3 days 00:21:10.588235294,1 days 08:24:36.923076923,1.289542,1.131397e-04,0.902179,1.562430,1.148352,1.449963
